In [ ]:
%pylab inline
import pandas as pd
import random

In [ ]:
SUBJECT = "RS06"
N_TRIALS = 20 # take the last so many trials from the training block
N_TARGETS  = 5 # how many targets to select

In [ ]:
inp = pd.read_csv('{}_recogAngles.csv'.format(SUBJECT))
ec_train_dir = inp['ec_train_dir']
ec_Day1Recall_dir = inp['ec_Day1Recall_dir']

In [ ]:
assert len(ec_train_dir)==N_TARGETS
assert len(ec_Day1Recall_dir)==N_TARGETS

In [ ]:
dat = pd.read_csv("{}_data.csv".format(SUBJECT)) # all the data for that subject
movs = list(dat[ (dat['sessionName']=='Day1Training') & (dat['trialtype']=='feedback') ]['handAngle'])[-N_TRIALS:]
assert len(movs)==N_TRIALS

In [ ]:
def iqr(ang, num_angle=5):
    """ Sample num_angle items from within the interquartile range of ang. Fancy function from Neeraj. """
    rand_ang = []
    q75, q50, q25 = np.percentile(ang, [75 ,50, 25])
    within_qr = [ a for a in ang if a>=q25 and a<=q75 ]
    print("{} values in IQR".format(len(within_qr)))
    rand_ang =  random.sample(list(within_qr), num_angle-1)
    rand_ang.append(q50)
    return rand_ang #, qr

Now we need to "predict" where they would move on day 2 recall.
For this, see the notebook `PredictionUsingVeryDeepNeuralNetwork`

In [ ]:
PREDICT_SLOPE = 0.3213691553011379 # from that other notebook with the fancy name

In [ ]:
ec_Day2Predict_dir = [ PREDICT_SLOPE*d for d in iqr(movs,N_TARGETS)]

# Some checks

In [ ]:
for i,(vals,lbl) in enumerate([ 
    (ec_train_dir,'ec_train_dir'),
    (ec_Day1Recall_dir,'ec_Day1Recall_dir'),
    (ec_Day2Predict_dir,'ec_Day2Predict_dir'),
]):
    plot([i]*len(vals),vals,'o',label=lbl)
legend()

In [ ]:
vals

# Make a recognition schedule from this.

In [ ]:
# Let's take all combinations of pairs from the three lists.
sources = [(ec_train_dir,'ec_train_dir'),
    (ec_Day1Recall_dir,'ec_Day1Recall_dir'),
    (ec_Day2Predict_dir,'ec_Day2Predict_dir')]

In [ ]:
biglist = []
for i,(valsP,lblP) in enumerate(sources):
    for j,(valsQ,lblQ) in enumerate(sources):
        if i<j:
            print(lblP,lblQ)
            for p in valsP:
                for q in valsQ:
                    flip = random.choice([True]) #[True,False])
                    if flip:
                        a,b,la,lb=p,q,lblP,lblQ
                    else:
                        a,b,la,lb=q,p,lblQ,lblP
                    biglist.append({'directionA':a,'directionB':b,'typeA':la,'typeB':lb})

In [ ]:
pd.DataFrame(biglist)

In [ ]:
pd.DataFrame(biglist).head()

In [ ]:
random.shuffle(biglist)

In [ ]:
for tr,b in enumerate(biglist):
    b['trial']=tr+1

In [ ]:
biglist = pd.DataFrame(biglist)
biglist.head()

In [ ]:
schedf = '{}_recognitionschedule.csv'.format(SUBJECT)
biglist.to_csv(schedf,index=False)